# Validating changes introduced in branch `10-density`

    
The version of `outputvalidation.py` at the time of writing returned `NARP` based on changes introduced in branch `10-density`. This notebook is a deep dive into why that happened. 

## Overview

Branch `10-density` made the following changes: 
1. introduced new density variables to the report output
    * This is the main change introduced in `10-density`
2. solidified user's option to drop `_raw_*` features from output
    * This secondary change impacts our work in this notebook


## A preliminary look at the data
We observe that there are 368 columns in `old` dataset that *do not* appear in `new`, and three columns in `new` that do not appear in `old`. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
old = pd.read_csv("../oldcomparetest.csv") # data before the change
new = pd.read_csv("../debugtest.csv")    # data after the change
new.head()

,RunID_vial,_out_crystalscore,_rxn_M_acid,_rxn_M_inorganic,_rxn_M_organic,_rxn_v1-M_acid,_rxn_v1-M_inorganic,_rxn_v1-M_organic,_rxn_organic-inchikey,_rxn_mixingtime1S,...,_raw_v1-M_UPHCENSIMPJEIS-UHFFFAOYSA-N_final,_raw_v1-M_XFYICZOIWSBQSK-UHFFFAOYSA-N_final,_raw_v1-M_YEJRWHAVMIAJKC-UHFFFAOYSA-N_final,_raw_v1-M_null_final,_raw_inchikey,_raw_smiles,_raw_molweight,_raw_smiles_standard,_raw_standard_molweight,_prototype_ECFP4_hexstring
0,2017-10-18T19_58_20.000000+00_00_LBL_A1,4.0,2.923979,0.888792,0.888764,2.923979,0.384735,0.384723,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,...,0.384723,0.0,3.336835,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.1,[NH3+]CCC1=CC=CC=C1,122.19,0000000000000008000000000000000100000000010000...
1,2017-10-18T19_58_20.000000+00_00_LBL_C1,4.0,4.779492,0.818854,0.818828,4.779492,0.354461,0.354449,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,...,0.354449,0.0,3.074264,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.1,[NH3+]CCC1=CC=CC=C1,122.19,0000000000000008000000000000000100000000010000...
2,2017-10-18T19_58_20.000000+00_00_LBL_E1,4.0,3.576726,0.864189,0.864161,3.576726,0.374085,0.374073,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,...,0.374073,0.0,3.244465,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.1,[NH3+]CCC1=CC=CC=C1,122.19,0000000000000008000000000000000100000000010000...
3,2017-10-18T19_58_20.000000+00_00_LBL_G1,4.0,3.576726,0.864189,0.864161,3.576726,0.374085,0.374073,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,...,0.374073,0.0,3.244465,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.1,[NH3+]CCC1=CC=CC=C1,122.19,0000000000000008000000000000000100000000010000...
4,2017-10-18T19_58_20.000000+00_00_LBL_B1,4.0,4.779492,0.818854,0.818828,4.779492,0.354461,0.354449,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,...,0.354449,0.0,3.074264,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.1,[NH3+]CCC1=CC=CC=C1,122.19,0000000000000008000000000000000100000000010000...


In [3]:
#  calculates the number of columns in the old dataset that do not appear in the new dataset
np.isin(old.columns.values, new.columns.values, invert=True).sum()

18

In [4]:
#  calculates the number of columns in the new dataset that do not appear in the old datset
np.isin(new.columns.values, old.columns.values, invert=True).sum()

61

## Columns missing from `new`

Here we can take a closer look at the columns that appeared in `old` but not `new`.

(Mike): We expected these to be the `_raw_*` columns, because of change 2. discussed in the **Overview**.
However, observe below that some `_feat_*` columns appear in this set, I'm not sure why. 

(Ian): The `_feat_*` columns have been removed from the perovskite features due to being terrible fits for the current dataset.  The target dataset might need to be updated to reflect the new feature set at the conclusion of validating and pushing to master.  I will make this change and the `_feat_*` columns should no longer appear (unless of course there are expected changes to the reporting of features in the updates being tested in a new branch)

In [5]:
dropped_cols = list(set(old.columns.values) - set(new.columns.values))
dropped_cols

['_feat_sol_ph8',
 '_feat_sol_ph12',
 '_feat_sol_ph7',
 '_feat_sol_ph10',
 '_feat_sol_ph11',
 '_feat_sol_ph2',
 '_feat_sol_ph5',
 '_feat_sol_ph0',
 '_feat_sol_ph9',
 '_feat_sol_ph3',
 '_raw_ECFP4_hexstring',
 '_feat_sol_ph1',
 '_feat_sol_ph13',
 '_feat_sol_ph4',
 '_feat_sol_ph6',
 '_feat_bpKa1',
 '_feat_sol_ph14',
 '_feat_fsp3']

In [6]:
dropped_feats = [col for col in dropped_cols if col[:4] != '_raw']

In [7]:
dropped_feats

['_feat_sol_ph8',
 '_feat_sol_ph12',
 '_feat_sol_ph7',
 '_feat_sol_ph10',
 '_feat_sol_ph11',
 '_feat_sol_ph2',
 '_feat_sol_ph5',
 '_feat_sol_ph0',
 '_feat_sol_ph9',
 '_feat_sol_ph3',
 '_feat_sol_ph1',
 '_feat_sol_ph13',
 '_feat_sol_ph4',
 '_feat_sol_ph6',
 '_feat_bpKa1',
 '_feat_sol_ph14',
 '_feat_fsp3']

In [8]:
len(dropped_cols)

18

In [9]:
len(dropped_feats)


17

In [10]:
list(set(old.columns.values) - set(new.columns.values))

['_feat_sol_ph8',
 '_feat_sol_ph12',
 '_feat_sol_ph7',
 '_feat_sol_ph10',
 '_feat_sol_ph11',
 '_feat_sol_ph2',
 '_feat_sol_ph5',
 '_feat_sol_ph0',
 '_feat_sol_ph9',
 '_feat_sol_ph3',
 '_raw_ECFP4_hexstring',
 '_feat_sol_ph1',
 '_feat_sol_ph13',
 '_feat_sol_ph4',
 '_feat_sol_ph6',
 '_feat_bpKa1',
 '_feat_sol_ph14',
 '_feat_fsp3']

## Columns added to `new`
These are the new columns added to the report output by branch `10-density`

In [11]:
new_cols = list(set(new.columns.values) - set(old.columns.values))

In [12]:
new_cols

['_raw_reagent_1_v1-conc_LLWRXQXPJMPHLR-UHFFFAOYSA-N',
 '_raw_reagent_3_v1-conc_YEJRWHAVMIAJKC-UHFFFAOYSA-N',
 '_raw_reagent_2_v1-conc_PPCHYMCMRUGLHR-UHFFFAOYSA-N',
 '_raw_reagent_4_v1-conc_null',
 '_raw_v1-mmol_XFYICZOIWSBQSK-UHFFFAOYSA-N_final',
 '_raw_v1-M_YEJRWHAVMIAJKC-UHFFFAOYSA-N_final',
 '_raw_reagent_2_v1-conc_RQQRAHKHDFPBMC-UHFFFAOYSA-L',
 '_raw_reagent_2_v1-conc_null',
 '_raw_v1-mmol_LLWRXQXPJMPHLR-UHFFFAOYSA-N_final',
 '_raw_reagent_3_v1-conc_CALQKRVFTWDYDG-UHFFFAOYSA-N',
 '_raw_v1-M_LLWRXQXPJMPHLR-UHFFFAOYSA-N_final',
 '_raw_reagent_2_v1-conc_FCTHQYIDLRRROX-UHFFFAOYSA-N',
 '_raw_reagent_2_v1-conc_XFYICZOIWSBQSK-UHFFFAOYSA-N',
 '_raw_reagent_6_v1-conc_null',
 '_raw_v1-mmol_YEJRWHAVMIAJKC-UHFFFAOYSA-N_final',
 '_raw_v1-M_IAZDPXIOMUYVGZ-UHFFFAOYSA-N_final',
 '_rxn_v1-M_acid',
 '_raw_reagent_6_v1-conc_BDAGIHXWWSANSR-UHFFFAOYSA-N',
 '_raw_reagent_0_v1-conc_YEJRWHAVMIAJKC-UHFFFAOYSA-N',
 '_raw_reagent_1_v1-conc_PPCHYMCMRUGLHR-UHFFFAOYSA-N',
 '_raw_v1-mmol_DMFMZFFIQRMJQZ-UHFFFAOY

## Dropping Mismatched Columns

Once we drop the columns such that the two dfs have the set intersection of their columns, we expect the dataframes to be equal. However, thats not *quite* what happens

In [13]:
old.drop(dropped_cols, axis=1, inplace=True)
old

,RunID_vial,_out_crystalscore,_rxn_M_acid,_rxn_M_inorganic,_rxn_M_organic,_rxn_organic-inchikey,_rxn_mixingtime1S,_rxn_mixingtime2S,_rxn_reactiontimeS,_rxn_stirrateRPM,...,_raw_M_LLWRXQXPJMPHLR-UHFFFAOYSA-N_final,_raw_M_PPCHYMCMRUGLHR-UHFFFAOYSA-N_final,_raw_M_RQQRAHKHDFPBMC-UHFFFAOYSA-L_final,_raw_M_UPHCENSIMPJEIS-UHFFFAOYSA-N_final,_raw_M_XFYICZOIWSBQSK-UHFFFAOYSA-N_final,_raw_inchikey,_raw_smiles,_raw_molweight,_raw_smiles_standard,_raw_standard_molweight
0,2017-10-18T19_58_20.000000+00_00_LBL_A1,4.0,2.923979,6.303724,0.888764,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,6.303724,0.888764,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
1,2017-10-18T19_58_20.000000+00_00_LBL_C1,4.0,4.779492,5.807694,0.818828,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,5.807694,0.818828,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
2,2017-10-18T19_58_20.000000+00_00_LBL_E1,4.0,3.576726,6.129227,0.864161,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,6.129227,0.864161,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
3,2017-10-18T19_58_20.000000+00_00_LBL_G1,4.0,3.576726,6.129227,0.864161,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,6.129227,0.864161,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
4,2017-10-18T19_58_20.000000+00_00_LBL_B1,4.0,4.779492,5.807694,0.818828,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,5.807694,0.818828,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
5,2017-10-18T19_58_20.000000+00_00_LBL_D1,4.0,4.194307,5.964130,0.840884,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,5.964130,0.840884,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
6,2017-10-18T19_58_20.000000+00_00_LBL_F1,4.0,4.194307,5.964130,0.840884,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,5.964130,0.840884,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
7,2017-10-18T19_58_20.000000+00_00_LBL_H1,4.0,5.266911,5.677393,0.800457,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,5.677393,0.800457,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
8,2017-10-18T19_58_20.000000+00_00_LBL_A2,4.0,2.923979,6.303724,0.888764,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,6.303724,0.888764,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
9,2017-10-18T19_58_20.000000+00_00_LBL_C2,4.0,4.779492,5.807694,0.818828,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,5.807694,0.818828,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19


In [14]:
new.drop(list(new_cols), axis=1, inplace=True)
new

,RunID_vial,_out_crystalscore,_rxn_M_acid,_rxn_M_inorganic,_rxn_M_organic,_rxn_organic-inchikey,_rxn_mixingtime1S,_rxn_mixingtime2S,_rxn_reactiontimeS,_rxn_stirrateRPM,...,_raw_M_LLWRXQXPJMPHLR-UHFFFAOYSA-N_final,_raw_M_PPCHYMCMRUGLHR-UHFFFAOYSA-N_final,_raw_M_RQQRAHKHDFPBMC-UHFFFAOYSA-L_final,_raw_M_UPHCENSIMPJEIS-UHFFFAOYSA-N_final,_raw_M_XFYICZOIWSBQSK-UHFFFAOYSA-N_final,_raw_inchikey,_raw_smiles,_raw_molweight,_raw_smiles_standard,_raw_standard_molweight
0,2017-10-18T19_58_20.000000+00_00_LBL_A1,4.0,2.923979,0.888792,0.888764,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.888792,0.888764,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
1,2017-10-18T19_58_20.000000+00_00_LBL_C1,4.0,4.779492,0.818854,0.818828,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.818854,0.818828,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
2,2017-10-18T19_58_20.000000+00_00_LBL_E1,4.0,3.576726,0.864189,0.864161,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.864189,0.864161,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
3,2017-10-18T19_58_20.000000+00_00_LBL_G1,4.0,3.576726,0.864189,0.864161,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.864189,0.864161,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
4,2017-10-18T19_58_20.000000+00_00_LBL_B1,4.0,4.779492,0.818854,0.818828,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.818854,0.818828,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
5,2017-10-18T19_58_20.000000+00_00_LBL_D1,4.0,4.194307,0.840911,0.840884,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.840911,0.840884,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
6,2017-10-18T19_58_20.000000+00_00_LBL_F1,4.0,4.194307,0.840911,0.840884,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.840911,0.840884,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
7,2017-10-18T19_58_20.000000+00_00_LBL_H1,4.0,5.266911,0.800483,0.800457,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.800483,0.800457,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
8,2017-10-18T19_58_20.000000+00_00_LBL_A2,4.0,2.923979,0.888792,0.888764,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.888792,0.888764,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19
9,2017-10-18T19_58_20.000000+00_00_LBL_C2,4.0,4.779492,0.818854,0.818828,UPHCENSIMPJEIS-UHFFFAOYSA-N,900.0,1200.0,9000.0,500.0,...,0.0,0.0,0.818854,0.818828,0.0,UPHCENSIMPJEIS-UHFFFAOYSA-N,C1=CC=C(C=C1)CC[NH3+].[I-],249.10,[NH3+]CCC1=CC=CC=C1,122.19


In [15]:
old.equals(new)

False

In [16]:
# A different way of comparing all values of the dataframe
(old == new).all().all()

False

## Dealing with `NaN`s
If we dig a little deeper into the differences between the two dataframes, it appears that `NaNs` are the culprit. 

If we just drop (few) rows that contain `NaNs`, everything looks good.

In [17]:
# gives us the indices of the rows and columns where mismatches are present 

mismatch_rows, mismatch_columns = list(map(lambda x: list(np.unique(x)), np.where(old != new)))
mismatch_columns

[1,
 3,
 10,
 83,
 86,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 113,
 116,
 118,
 167,
 212,
 213,
 214,
 215,
 216,
 221,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 262,
 263,
 265,
 268,
 270,
 273,
 278,
 280,
 281,
 282,
 283,
 284,
 290,
 291,
 292,
 293,
 294,
 304,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 337,
 340,
 342,
 357,
 358,
 359,
 363,
 364,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 380,
 383,
 385,
 410]

In [18]:
old.iloc[mismatch_rows, mismatch_columns]

,_out_crystalscore,_rxn_M_inorganic,_rxn_temperatureC_actual_bulk,_raw_mmol_RQQRAHKHDFPBMC-UHFFFAOYSA-L_final,_raw_modelname,_raw_reagent_0_chemicals_1_InChIKey,_raw_reagent_0_chemicals_1_actual_amount,_raw_reagent_0_chemicals_1_actual_amount_units,_raw_reagent_0_chemicals_1_nominal_amount,_raw_reagent_0_chemicals_1_nominal_amount_units,...,_raw_reagent_6_chemicals_1_nominal_amount_units,_raw_reagent_6_chemicals_2_InChIKey,_raw_reagent_6_chemicals_2_actual_amount,_raw_reagent_6_chemicals_2_actual_amount_units,_raw_reagent_6_chemicals_2_nominal_amount,_raw_reagent_6_chemicals_2_nominal_amount_units,_raw_reagent_6_instructions_0_duration,_raw_reagent_6_instructions_0_stir,_raw_reagent_6_instructions_0_temperature,_raw_M_RQQRAHKHDFPBMC-UHFFFAOYSA-L_final
0,4.0,6.303724,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.303724
1,4.0,5.807694,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.807694
2,4.0,6.129227,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.129227
3,4.0,6.129227,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.129227
4,4.0,5.807694,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.807694
5,4.0,5.964130,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.964130
6,4.0,5.964130,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.964130
7,4.0,5.677393,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.677393
8,4.0,6.303724,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.303724
9,4.0,5.807694,77.0,3.542693,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.807694


In [19]:
new.iloc[mismatch_rows, mismatch_columns]

,_out_crystalscore,_rxn_M_inorganic,_rxn_temperatureC_actual_bulk,_raw_mmol_RQQRAHKHDFPBMC-UHFFFAOYSA-L_final,_raw_modelname,_raw_reagent_0_chemicals_1_InChIKey,_raw_reagent_0_chemicals_1_actual_amount,_raw_reagent_0_chemicals_1_actual_amount_units,_raw_reagent_0_chemicals_1_nominal_amount,_raw_reagent_0_chemicals_1_nominal_amount_units,...,_raw_reagent_6_chemicals_1_nominal_amount_units,_raw_reagent_6_chemicals_2_InChIKey,_raw_reagent_6_chemicals_2_actual_amount,_raw_reagent_6_chemicals_2_actual_amount_units,_raw_reagent_6_chemicals_2_nominal_amount,_raw_reagent_6_chemicals_2_nominal_amount_units,_raw_reagent_6_instructions_0_duration,_raw_reagent_6_instructions_0_stir,_raw_reagent_6_instructions_0_temperature,_raw_M_RQQRAHKHDFPBMC-UHFFFAOYSA-L_final
0,4.0,0.888792,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.888792
1,4.0,0.818854,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.818854
2,4.0,0.864189,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.864189
3,4.0,0.864189,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.864189
4,4.0,0.818854,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.818854
5,4.0,0.840911,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.840911
6,4.0,0.840911,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.840911
7,4.0,0.800483,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800483
8,4.0,0.888792,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.888792
9,4.0,0.818854,77.0,0.499501,NaN,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.818854


In [20]:
old_dropnans = old.drop(mismatch_rows)
new_dropnans = new.drop(mismatch_rows)

Note that [pd.df.equals](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.equals.html) still returns `False`, but checking equality 'by hand' returns `True`. I am inclined to trust the 'by hand' check, since pandas can be quite finnickey

In [21]:
old_dropnans.equals(new_dropnans)

False

In [22]:
(old_dropnans == new_dropnans).all().all()

True